# CSC411 Assignment 3

## Helper Functions for Part 2

In [3]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as mpimg
import os
import urllib
# Constants
# -----------------------------------------------------------------------------

In [4]:
# Functions used in multiple parts
# -----------------------------------------------------------------------------
def getNews(seed = 0):
    
    fakeFile = open("clean_fake.txt",'r')
    fakeNews = fakeFile.read().splitlines()
    fakeFile.close()
    realFile = open("clean_real.txt",'r')
    realNews = realFile.read().splitlines()
    realFile.close()
    
    np.random.shuffle(fakeNews)
    np.random.shuffle(realNews)
    
    return fakeNews, realNews

In [5]:
# test
fakeNews, realNews = getNews()


In [6]:
def dataSetSplit(fakeNews, realNews,seed = 0):
    
    trainingSet ={'real': realNews[:int(0.7*len(realNews))], 'fake':fakeNews[:int(0.7*len(fakeNews))]} 
    validationSet = {'real':realNews[int(0.7*len(realNews)):int(0.85*len(realNews))], 'fake':fakeNews[int(0.7*len(fakeNews)):int(0.85*len(fakeNews))]}
    testSet = {'real': realNews[int(0.85*len(realNews)):] , 'fake': fakeNews[int(0.85*len(fakeNews)):]}

    return trainingSet, validationSet, testSet

In [7]:
# test
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)

In [8]:
# test
trainingSet

{'fake': ['comment on trump election will be the biggest f you in history by dr eowyn',
  'rising up to the trump moment we are the ones we ve been waiting for',
  'trump invigorates new american volk by formulating u einsatzgruppen',
  'did trump make a secret deal not to prosecute hillary in exchange for her concession',
  'shock video hillary supporters attack homeless woman protecting trump star',
  'the intercept outs neocon democrat smear against trump as putin puppet eric zuesse',
  'homeless trump supporter posts as guard for trump hollywood star liberals lose it',
  'trump has gotten the republican party sued for trying to intimidate voters',
  'cbs 60 minutes withheld trump appeal to stop attacking minorities and ignored reports of attacks on trump supporters',
  'dr david duke with mark collett of the uk collett explains why duke trump victories would change politics forever',
  'netanyahu calls trump true friend of israel likud party pols call for u embassy move to jerusale

In [9]:
def count(trainingSet, validationSet, testSet):
    trainingCount = {'real':{}, 'fake':{}}
    validationCount = {'real':{}, 'fake':{}}
    testCount = {'real':{}, 'fake':{}}
    
    for rof in ['real','fake']:
        for news in trainingSet[rof]:
            for word in set(news.split(' ')):
                if not word in trainingCount[rof].keys():
                    trainingCount[rof][word] = 1
                else:
                    trainingCount[rof][word] += 1
        
        for news in validationSet[rof]:
            for word in set(news.split(' ')):
                if not word in validationCount[rof].keys():
                    validationCount[rof][word] = 1
                else:
                    validationCount[rof][word] += 1
        
        for news in testSet[rof]:
            for word in set(news.split(' ')):
                if not word in testCount[rof].keys():
                    testCount[rof][word] = 1
                else:
                    testCount[rof][word] += 1

    trainingRealCount = len(trainingSet['real'])
    trainingFakeCount = len(trainingSet['fake'])
    validationRealCount = len(validationSet['real'])
    validationFakeCount = len(validationSet['fake'])
    testRealCount = len(testSet['real'])
    testFakeCount = len(testSet['fake'])
    
    return trainingRealCount,trainingFakeCount,validationRealCount,validationFakeCount,testRealCount,testFakeCount, trainingCount, validationCount, testCount


In [10]:
# test
trainingRealCount,trainingFakeCount,validationRealCount,validationFakeCount,testRealCount,testFakeCount, trainingCount, validationCount, testCount = count(trainingSet, validationSet, testSet)

In [11]:
# test
trainingCount

{'fake': {'pardon': 3,
  'hats': 1,
  'pide': 1,
  'colleges': 1,
  'four': 1,
  'debunks': 1,
  'protest': 5,
  'hating': 1,
  'battleground': 2,
  'hate': 6,
  'assembled': 1,
  'whose': 1,
  'votes': 11,
  'voter': 22,
  'pointing': 2,
  'dazu': 1,
  'tweet': 1,
  'cashin': 1,
  'edward': 2,
  'voted': 1,
  'under': 2,
  'teaching': 1,
  'sway': 1,
  'regrets': 1,
  'divergent': 1,
  'risk': 1,
  'populist': 1,
  'smack': 1,
  'every': 7,
  'politician': 1,
  'bringing': 1,
  'school': 2,
  'snowflakes': 1,
  'reductress': 1,
  'indefinitely': 1,
  'red': 2,
  'revolted': 1,
  'deplorable': 3,
  'clothes': 1,
  'uncovers': 2,
  'force': 2,
  'leaders': 1,
  '10m': 1,
  'warns': 7,
  'pilger': 1,
  '270': 2,
  'crave': 5,
  'street': 2,
  '49': 1,
  'trumpprotest': 1,
  'n': 4,
  'draintheswamp': 1,
  'machines': 6,
  'even': 6,
  'army': 1,
  'brexit': 4,
  'nwo': 1,
  'reopens': 1,
  'religious': 1,
  'beaten': 5,
  'fingers': 1,
  'liberty': 10,
  'children': 3,
  'daesh': 1,
  'u

In [12]:
def probability(trainingRealCount,trainingFakeCount, trainingCount, m, pHat):
    trainingProbability = {'real': {}, 'fake': {}}
    for word in trainingCount['real'].keys():
        trainingProbability['real'][word] = (trainingCount['real'][word]+m*pHat)/float(trainingRealCount+m)
    for word in trainingCount['fake'].keys():
        trainingProbability['fake'][word] = (trainingCount['fake'][word]+m*pHat)/float(trainingFakeCount+m)
    
    realProbability = float(trainingRealCount)/(trainingRealCount+trainingFakeCount)
    fakeProbability = float(trainingFakeCount)/(trainingRealCount+trainingFakeCount)
    
    return trainingProbability, realProbability, fakeProbability

In [38]:
# test
mReal = 0.1
pHatReal = 0.0001
mFake = 0.1
pHatFake = 0.0001
trainingProbability, realProbability, fakeProbability = probability(trainingRealCount,trainingFakeCount, trainingCount, m, pHat)

In [ ]:
# test
trainingProbability

In [14]:
def getAccuracy(trainingProbability,realProbability, fakeProbability, targetSet, m, pHat):
    accuracy = 0
    for rof in ['real', 'fake']:
        for numNews in range(len(targetSet[rof])):
            logProb = 0   
            for word in trainingProbability['real'].keys():
                if word in targetSet[rof][numNews].split(' '):
                    logProb += log(trainingProbability['real'][word])
                else:
                    logProb += log(1-trainingProbability['real'][word])
            for word in targetSet[rof][numNews].split(' '):
                if not word in trainingProbability['real'].keys():
                    logProb += log(pHat)
            logProb += log(realProbability)
            realProb = logProb

            logProb = 0 
            for word in trainingProbability['fake'].keys():
                if word in targetSet[rof][numNews].split(' '):
                    logProb += log(trainingProbability['fake'][word])
                else:
                    logProb += log(1-trainingProbability['fake'][word])
            for word in targetSet[rof][numNews].split(' '):
                if not word in trainingProbability['fake'].keys():
                    logProb += log(pHat)
            logProb += log(fakeProbability)
            fakeProb = logProb
            
            if rof == 'real':
                if realProb > fakeProb:
                    accuracy += 1
            else:
                if realProb < fakeProb:
                    accuracy += 1
                    
    return accuracy / (float(len(targetSet['fake']))+float(len(targetSet['real'])))


In [15]:
# test
np.random.seed(0)

fakeNews, realNews = getNews()
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
trainingRealCount,trainingFakeCount,validationRealCount, \
validationFakeCount,testRealCount,testFakeCount, trainingCount, \
validationCount, testCount = count(trainingSet, validationSet, testSet)

m = 0.1
pHat = 0.0001
trainingProbability, realProbability, fakeProbability = \
probability(trainingRealCount,trainingFakeCount, trainingCount, m,pHat)

getAccuracy(trainingProbability,realProbability, fakeProbability, validationSet, m, pHat)

0.8795918367346939

## Part 2

In [16]:
def part2(seed = 0):
    np.random.seed(seed)

    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    trainingRealCount,trainingFakeCount,validationRealCount, \
    validationFakeCount,testRealCount,testFakeCount, trainingCount, \
    validationCount, testCount = count(trainingSet, validationSet, testSet)
    
    m = 0.1
    pHat = 0.0001
    trainingProbability, realProbability, fakeProbability = \
    probability(trainingRealCount,trainingFakeCount, trainingCount, m,pHat)

    print getAccuracy(trainingProbability,realProbability, fakeProbability, validationSet, m, pHat)
    print getAccuracy(trainingProbability,realProbability, fakeProbability, testSet, m, pHat)
    print getAccuracy(trainingProbability,realProbability, fakeProbability, trainingSet, m, pHat)

## Helper Functions for Part 4

In [144]:
def generateWordBase(trainingSet):
    wordBase = []
    for rof in trainingSet.keys():
        for news in trainingSet[rof]:
            for word in news.split(' '):
                wordBase.append(word)
    return list(set(wordBase))

In [145]:
# test
np.random.seed(0)
fakeNews, realNews = getNews()
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
print generateWordBase(trainingSet)

['pardon', 'hats', 'pide', 'bosses', 'four', 'obstruction', 'protest', 'universe', 'controversial', 'aides', 'battleground', 'hate', 'assembled', 'marching', 'stinks', 'looking', 'votes', 'voter', 'dazu', 'paris', 'tweet', 'tonya', 'cashin', 'amtsantritt', 'activist', 'voted', 'kidman', 'under', 'teaching', 'condemns', 'whack', 'divergent', 'softener', 'evangelical', 'risk', 'dismal', '6', 'rise', 'populist', 'shoot', 'advisors', 'mania', 'every', 'jack', 'govern', 'affect', 'exporters', 'disneys', 'trumpisms', 'tickets', 'school', 'publicly', 'metals', 'called', 'skills', 'supports', 'companies', 'solution', 'piling', 'pardons', 'red', 'unhinged', 'deplorable', 'heading', 'triumph', 'clothes', 'jeff', 'achievements', 'force', 'leaders', 'tired', 'miller', 'warns', 'guidelines', 'bridgitte', 'holocaust', 'chef', 'talks', 'second', 'street', '49', 'streep', 'n', 'loathing', 'seth', 'machines', 'monster', 'blue', 'army', 'brexit', 'nwo', 'hide', 'reopens', 'religious', 'horcruxes', 'beat

In [146]:
def generateData(wordBase, targetSet):
    xy = np.zeros([len(wordBase)+2, (len(targetSet['real'])+len(targetSet['fake']))])
    n = 0
    for rof in ['real', 'fake']:
        for news in targetSet[rof]:
            for word in news:
                if word in wordBase:
                    xy[wordBase.index(word), n] = 1
            if rof == 'real':
                xy[-1, n] = 1
            n += 1
    xy[-2,:] = 1
    return xy

In [147]:
# test
training_xy = generateData(wordBase, trainingSet)

In [149]:
def sigmoidForward(y):
    return 1/(1+exp(-y))

def sigmoidBackward(y,output):
    return output-y

def fcForward(x,W):
    return np.matmul(W.T,x)

def fcBackward(dy,x):
    dW = np.matmul(x,dy.T)
    return dW
    
def forward(x, W):
    L1 = fcForward(x,W)
    output = sigmoidForward(L1)
    return output

def backward(y,output,x):
    dy = sigmoidBackward(y,output)
    dW = fcBackward(dy,x)
    return dW

def NLL(y, output):
    return -sum(y*log(output)+(1-y)*log((1-output)))

In [158]:
def grad_descent(f, df, loss, x, y, init_t,ada_learning_rate = True, \
                 tx=[], ty=[], figure_name = 'default',momentum = True, damping = 0.9, alpha=0.0001, \
                 max_iter=1000, EPS = 1e-5, zoom_in = 25):
    v = np.zeros((init_t.shape[0],init_t.shape[1])) #used for momentum
    
    prev_t = init_t-10*EPS
    t = init_t.copy()
    
    currloss = loss(y, f(x,t)) # used for adaptive alpha
    curr_alpha = alpha # used for adaptive alpha
    
    iter  = 0
    num_iter = []
    performance_training = []
    performance_test = []
    
    while norm(t - prev_t) >  EPS and iter < max_iter:
        
        #normal gradient descent
        prev_t = t.copy()
        if momentum:
            v = damping*v+curr_alpha*df(y, f(x,t), x)
            t -= v
        else:
            t -= curr_alpha*df(y, f(x,t), x)
        
        if iter % 5 == 0:
            print "Iter", iter
            if currloss < loss(y, f(x,t)) and ada_learning_rate:
                curr_alpha = curr_alpha/2
            currloss = loss(y, f(x,t))
            print currloss
        
        iter += 1
    
    return t

In [167]:
# test
x = np.array([[1,1],[2,2],[3,2]])
y = np.array([[1,0]])
W = np.array([[1],[2],[3]])
output = forward(x, W)
a = backward(y,output,x)
a.shape[0]==3

True

In [241]:
def performance(x, y, W, print_output = True,report = "test set"):
    output = forward(x,W)
    result = 0.
    for i in range(x.shape[1]):
        #print output[0,i]
        #print y[i]
        if (output[0,i]>0.5 and y[i]==1) or (output[0,i]<0.5 and y[i]==0):
            result+=1
            
    if print_output:
        print "Performance on "+report+ ": " +str(result)+"/"+str(x.shape[1])+"\n"
    
    return result/x.shape[1]

## Part 4


In [236]:
def part4(seed = 0):
    np.random.seed(seed)
    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    wordBase = generateWordBase(trainingSet)
    
    training_xy = generateData(wordBase, trainingSet)
    validation_xy = generateData(wordBase, validationSet)
    test_xy = generateData(wordBase, testSet)
    
    W = np.random.normal(0.,0.01,[len(wordBase)+1,1])
    trainedW = grad_descent(forward, backward, NLL, training_xy[:-1,:], training_xy[-1,:], W,momentum = False, max_iter=2000, alpha=0.01,figure_name = "part4f1", tx=test_xy[:-1,:], ty=test_xy[-1,:])
    
    return training_xy, validation_xy, test_xy, trainedW

In [242]:
training_xy, validation_xy, test_xy, trainedW = part4()
#performance(test_xy[:-1,:], test_xy[-1,:], trainedW, print_output = True,report = "test set")

Iter 0
27657.333103687004
Iter 5
6017.9214634067985
Iter 10
27253.79875259378
Iter 15
9628.985736761522
Iter 20
12233.18478390304
Iter 25
4707.122589575418
Iter 30
4953.096895450551
Iter 35
2761.9345933605664
Iter 40
2607.308429342117
Iter 45
2459.2958000679387
Iter 50
2318.5675537261195
Iter 55
2185.8547704972752
Iter 60
2061.925748004392
Iter 65
1947.550644243569
Iter 70
1843.450738445424
Iter 75
1750.232533575044
Iter 80
1668.3120555311662
Iter 85
1597.837446092284
Iter 90
1538.6210912394704
Iter 95
1490.100087042873
Iter 100
1451.3479610806228
Iter 105
1421.1495857618693
Iter 110
1398.1271436282004
Iter 115
1380.8836912209285
Iter 120
1368.1309564659355
Iter 125
1372.9130691525106
Iter 130
1355.2911045617323
Iter 135
1351.939109630323
Iter 140
1349.2002691922348
Iter 145
1346.8383400973428
Iter 150
1344.7932160731116
Iter 155
1343.016179540765
Iter 160
1341.4660781999087
Iter 165
1340.1082374123287
Iter 170
1338.9135188990097
Iter 175
1337.8574824238162
Iter 180
1336.919650698229
I

Iter 1485
1323.1090949790846
Iter 1490
1323.1033278737661
Iter 1495
1323.0975869678164
Iter 1500
1323.0918720854584
Iter 1505
1323.0861830525153
Iter 1510
1323.080519696388
Iter 1515
1323.074881846038
Iter 1520
1323.069269331966
Iter 1525
1323.0636819861934
Iter 1530
1323.0581196422436
Iter 1535
1323.0525821351234
Iter 1540
1323.047069301305
Iter 1545
1323.0415809787091
Iter 1550
1323.0361170066858
Iter 1555
1323.0306772259985
Iter 1560
1323.0252614788074
Iter 1565
1323.0198696086525
Iter 1570
1323.014501460437
Iter 1575
1323.0091568804123
Iter 1580
1323.0038357161613
Iter 1585
1322.998537816584
Iter 1590
1322.9932630318815
Iter 1595
1322.9880112135415
Iter 1600
1322.9827822143232
Iter 1605
1322.9775758882436
Iter 1610
1322.972392090563
Iter 1615
1322.9672306777707
Iter 1620
1322.9620915075718
Iter 1625
1322.9569744388727
Iter 1630
1322.9518793317695
Iter 1635
1322.9468060475328
Iter 1640
1322.9417544485968
Iter 1645
1322.9367243985448
Iter 1650
1322.931715762098
Iter 1655
1322.9267284

In [244]:
performance(test_xy[:-1,:], test_xy[-1,:], trainedW, print_output = True,report = "test set")

Performance on test set: 332.0/491



0.6761710794297352

In [245]:
performance(validation_xy[:-1,:], validation_xy[-1,:], trainedW, print_output = True,report = "test set")

Performance on test set: 338.0/490



0.689795918367347

In [246]:
performance(training_xy[:-1,:], training_xy[-1,:], trainedW, print_output = True,report = "test set")

Performance on test set: 1625.0/2285



0.7111597374179431